In [ ]:
from urllib.request import urlretrieve
import zipfile, os
import pandas as pd
import numpy as np
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_train.csv" "/content"
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_ICM_title_abstract.csv" "/content"
!cp  "drive/My Drive/Colab Notebooks/Recommender System/data_target_users_test.csv" "/content"
!unzip '/content/drive/My Drive/Colab Notebooks/Recommender System/RecSys_Course_AT_PoliMi-master.zip'
!cp  "/content/RecSys_Course_AT_PoliMi-master/Base/Evaluation/Evaluator.py" "/content"

sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Data_manager/split_functions')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master')



URM_all_dataframe = pd.read_csv("/content/data_train.csv", 
                                sep=",", 
                                names=["user_id", "item_id", "interaction"],
                                header=0,
                                dtype ={"row": np.int32,
                                "col": np.int32,
                                "data": np.int32})

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]


ICM_dataframe = pd.read_csv("/content/data_ICM_title_abstract.csv", 
                            sep=",", 
                            names=["item_id", "feature_id", "importance"],
                            header=0,
                            dtype={"row": np.int32,
                              "col": np.int32,
                              "data": np.float})


ICM_dataframe.columns = ["item_id", "feature_id", "importance"]

URM_all_dataframe_old = URM_all_dataframe
ICM_dataframe_old = ICM_dataframe

In [ ]:
item_original_ID_to_index_dict = {}
map_new_into_old = {}

for item_id in URM_all_dataframe["item_id"].unique():
    item_original_ID_to_index_dict[item_id] = len(item_original_ID_to_index_dict)
    map_new_into_old[item_original_ID_to_index_dict[item_id]] = item_id

print("Unique item_id in the URM are {}".format(len(item_original_ID_to_index_dict)))
    
for item_id in ICM_dataframe["item_id"].unique():
    if item_id not in item_original_ID_to_index_dict:
        item_original_ID_to_index_dict[item_id] = len(item_original_ID_to_index_dict)
        
print("Unique item_id in the URM and ICM are {}".format(len(item_original_ID_to_index_dict)))




feature_original_ID_to_index_dict = {}

for feature_id in ICM_dataframe["feature_id"].unique():
    feature_original_ID_to_index_dict[feature_id] = len(feature_original_ID_to_index_dict)

print("Unique feature_id in the URM are {}".format(len(feature_original_ID_to_index_dict)))




URM_all_dataframe["item_id"] = [item_original_ID_to_index_dict[item_original] for item_original in 
                                      URM_all_dataframe["item_id"].values]



                                      
ICM_dataframe["item_id"] = [item_original_ID_to_index_dict[item_original] for item_original in 
                                      ICM_dataframe["item_id"].values]

ICM_dataframe["feature_id"] = [feature_original_ID_to_index_dict[feature_original] for feature_original in 
                                      ICM_dataframe["feature_id"].values]

Unique item_id in the URM are 24896
Unique item_id in the URM and ICM are 25975
Unique feature_id in the URM are 19998


In [ ]:
import scipy.sparse as sps
import numpy as np

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(item_original_ID_to_index_dict)
n_features = len(feature_original_ID_to_index_dict)

URM_all = sps.csr_matrix((URM_all_dataframe["interaction"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))


ICM_all_importance = sps.csr_matrix((ICM_dataframe["importance"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all = sps.csr_matrix((np.ones(len(ICM_dataframe["item_id"].values)), 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all.data = np.ones_like(ICM_all.data)

**TRAIN/TEST SPLIT**

In [ ]:
import sys
sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Data_manager/split_functions')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master')
sys.path.append('/content/RecSys_Course_AT_PoliMi-master/Utils')


from Evaluator import EvaluatorHoldout
from split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)
URM_train_validation = URM_train + URM_validation

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

**HYPERPARAMATER TUNING WITH BAYESIAN**

In [ ]:
pip install h5py scikit-optimize

In [ ]:
import skopt
from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
import os
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender

def tune(i, evaluator_validation, evaluator_test):
      

      hyperparameters_range_dictionary = {}
      hyperparameters_range_dictionary["topK"] = Integer(10, 1000)
      hyperparameters_range_dictionary["alpha"] = Real(low = 0.1, high = 1, prior = 'uniform')
      hyperparameters_range_dictionary["beta"] = Real(low = 0, high = 0.1, prior = 'uniform')
      hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])




      recommender_class = RP3betaRecommender

      parameterSearch = SearchBayesianSkopt(recommender_class,
                                      evaluator_validation=evaluator_validation,
                                      evaluator_test=evaluator_test)



        
      recommender_input_args = SearchInputRecommenderArgs(
          CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
          CONSTRUCTOR_KEYWORD_ARGS = {},
          FIT_POSITIONAL_ARGS = [],
          FIT_KEYWORD_ARGS = {}
      )




      recommender_input_args_last_test = SearchInputRecommenderArgs(
          CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
          CONSTRUCTOR_KEYWORD_ARGS = {},
          FIT_POSITIONAL_ARGS = [],
          FIT_KEYWORD_ARGS = {}
      )




      output_folder_path = "/content/result_experiments/model_" + str(i)

      # If directory does not exist, create
      if not os.path.exists(output_folder_path):
          os.makedirs(output_folder_path)
          
      n_cases = 100
      n_random_starts = int(n_cases*0.2)
      metric_to_optimize = "MAP"




      parameterSearch.search(recommender_input_args,
                            recommender_input_args_last_test = recommender_input_args_last_test,
                            parameter_search_space = hyperparameters_range_dictionary,
                            n_cases = n_cases,
                            n_random_starts = n_random_starts,
                            save_model = "no",
                            output_folder_path = output_folder_path,
                            output_file_name_root = recommender_class.RECOMMENDER_NAME,
                            metric_to_optimize = metric_to_optimize,
                            )

**USER-WISE HYBRID**

In [ ]:
profile_length = np.ediff1d(URM_train.indptr)

#gruppi in base a interazioni: 
#da 0 a 2 -->gruppo 0
#da 3 a 5 -->gruppo 1
#ecc
ranges = [0, 3, 6, 9, 16,  51, 201, 1133]
groups = {} 
for i in range(len(ranges) -1):
  groups[i] = []


#riempio i gruppi con id utenti
for i in range(len(profile_length)):
  cont = 0
  for j in ranges:
    if profile_length[i] >= j:
      cont = cont +1
    else: 
      groups[cont-1].append(i)
      break  



In [ ]:
profile_length = np.ediff1d(URM_train.indptr)
print("PROFILE LENGTH: " + str(len(profile_length)))
block_size = int(len(profile_length)*0.01)
print("BLOCK SIZE: " + str(block_size))
#indice degli utenti in ordine crescente di numero di interazioni
sorted_users = np.argsort(profile_length)

for group_id in range(0, 101):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, average p.len {:.2f}, min {}, max {}".format(group_id, 
        users_in_group_p_len.mean(), users_in_group_p_len.min(), users_in_group_p_len.max()))



In [ ]:
for group in groups.values():
  print(group)

[0, 2, 3, 6, 8, 12, 13, 14, 16, 19, 24, 32, 34, 35, 39, 44, 45, 54, 55, 56, 58, 59, 67, 69, 70, 80, 85, 86, 89, 95, 96, 102, 105, 110, 113, 118, 120, 131, 132, 136, 138, 140, 144, 147, 152, 153, 154, 161, 162, 166, 167, 168, 169, 170, 171, 173, 175, 176, 178, 185, 188, 194, 196, 206, 212, 213, 218, 220, 223, 229, 232, 235, 239, 243, 248, 251, 254, 255, 258, 259, 263, 265, 266, 267, 268, 269, 270, 274, 279, 282, 285, 287, 290, 291, 293, 294, 300, 306, 309, 310, 313, 320, 321, 324, 326, 330, 338, 342, 343, 347, 348, 351, 354, 355, 356, 357, 363, 364, 366, 368, 373, 376, 381, 382, 386, 393, 403, 405, 407, 410, 412, 416, 422, 425, 426, 427, 428, 429, 431, 438, 442, 443, 445, 450, 454, 458, 460, 462, 464, 466, 473, 475, 482, 486, 488, 494, 506, 509, 515, 520, 522, 523, 530, 531, 533, 536, 538, 541, 547, 548, 549, 552, 556, 558, 559, 564, 568, 572, 575, 578, 582, 585, 586, 593, 598, 599, 600, 603, 604, 605, 606, 612, 614, 617, 618, 621, 622, 624, 632, 634, 637, 642, 648, 649, 653, 656, 657, 

In [ ]:
from Base.DataIO import DataIO
from GraphBased.RP3betaRecommender import RP3betaRecommender

MAP_itemKNNCF_per_group = []
MAP_itemKNNCBF_per_group = []
MAP_pureSVD_per_group = []
MAP_topPop_per_group = []

cutoff = 10
models = {}
evaluators_validation = {}
evaluators_test = {}
cont = 0

for group in groups.values(): 
  
    users_not_in_group_flag = np.isin(sorted_users, group, invert = True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    

    #evaluators
    evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users = users_not_in_group)
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users = users_not_in_group)

    evaluators_validation[cont] = evaluator_validation
    evaluators_test[cont] = evaluator_test
    

    print("Tuning group: " +str(cont))
    tune(cont, evaluator_validation, evaluator_test)    



    output_folder_path = "/content/result_experiments/model_" + str(cont) 
    recommender_class = RP3betaRecommender

    data_loader = DataIO(folder_path = output_folder_path)
    search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
    best_parameters_RP3beta = search_metadata["hyperparameters_best"]

    print("Modello " + str(cont))
    model = RP3betaRecommender(URM_train)
    model.fit(**best_parameters_RP3beta)

    models[cont] = model
    cont = cont + 1

  
    
 
    # results, _ = evaluator_test.evaluateRecommender(pureSVD)
    # MAP_pureSVD_per_group.append(results[cutoff]["MAP"])

    # results, _ = evaluator_test.evaluateRecommender(itemKNNCBF)
    # MAP_itemKNNCBF_per_group.append(results[cutoff]["MAP"])

    # results, _ = evaluator_test.evaluateRecommender(topPop)
    # MAP_topPop_per_group.append(results[cutoff]["MAP"])

In [ ]:
results, _ = evaluators_validation[6].evaluateRecommender(models[6])
results

EvaluatorHoldout: Processed 21 ( 100.00% ) in 0.04 sec. Users per second: 591


{10: {'ARHR': 0.5658919123204836,
  'AVERAGE_POPULARITY': 0.09756534192624418,
  'COVERAGE_ITEM': 0.006236766121270452,
  'COVERAGE_ITEM_CORRECT': 0.0013089509143407122,
  'COVERAGE_USER': 1.0,
  'COVERAGE_USER_CORRECT': 0.8571428571428571,
  'DIVERSITY_GINI': 0.0050460498335459835,
  'DIVERSITY_HERFINDAHL': 0.9917460317460317,
  'DIVERSITY_MEAN_INTER_LIST': 0.9633333333333334,
  'F1': 0.03404640201270233,
  'HIT_RATE': 1.7142857142857142,
  'MAP': 0.08469576719576719,
  'MRR': 0.3833900226757369,
  'NDCG': 0.04197349394893363,
  'NOVELTY': 0.004660397433281091,
  'PRECISION': 0.17142857142857149,
  'PRECISION_RECALL_MIN_DEN': 0.17142857142857149,
  'RECALL': 0.018900009900434372,
  'ROC_AUC': 0.5140513983371127,
  'SHANNON_ENTROPY': 7.170726634392705}}

**ALLENA I MODELLI SU TRAIN + VALID**

In [ ]:

finalModels = {}
cont = 0
for group in groups.values(): 
   
    output_folder_path = "/content/result_experiments/model_" + str(cont) 
    recommender_class = RP3betaRecommender

    data_loader = DataIO(folder_path = output_folder_path)
    search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
    best_parameters_RP3beta = search_metadata["hyperparameters_best"]

    print("Modello " + str(cont))
    model = RP3betaRecommender(URM_train_validation)
    model.fit(**best_parameters_RP3beta)

    finalModels[cont] = model
    cont = cont + 1

  

In [ ]:
results, _ = evaluators_test[2].evaluateRecommender(finalModels[6])
results

EvaluatorHoldout: Processed 682 ( 100.00% ) in 0.36 sec. Users per second: 1886


{10: {'ARHR': 0.10815004887585537,
  'AVERAGE_POPULARITY': 0.066948852854925,
  'COVERAGE_ITEM': 0.15345524542829644,
  'COVERAGE_ITEM_CORRECT': 0.006198267564966314,
  'COVERAGE_USER': 0.6705998033431662,
  'COVERAGE_USER_CORRECT': 0.14749262536873156,
  'DIVERSITY_GINI': 0.09848542801069698,
  'DIVERSITY_HERFINDAHL': 0.9992624762428943,
  'DIVERSITY_MEAN_INTER_LIST': 0.9940823611990302,
  'F1': 0.04330909969028012,
  'HIT_RATE': 0.2595307917888563,
  'MAP': 0.055276992272960034,
  'MRR': 0.09875366568914959,
  'NDCG': 0.08488566753976157,
  'NOVELTY': 0.005118317346769598,
  'PRECISION': 0.02595307917888562,
  'PRECISION_RECALL_MIN_DEN': 0.13074291300097748,
  'RECALL': 0.13074291300097748,
  'ROC_AUC': 0.1386823302145882,
  'SHANNON_ENTROPY': 11.404496939232615}}

**SUBMISSION**

In [ ]:
#3 min 44 sec
users = pd.read_csv("/content/data_target_users_test.csv", 
                       #sep=",", 
                       names=["user_id"],
                       header=0,
                       dtype={"user_id": np.int32,
                              })
utenti = users.user_id.unique()
print(utenti)
len(utenti)


def train_on_all():
  submissionModels = {}
  cont = 0
  for group in groups.values(): 
   
    output_folder_path = "/content/result_experiments/model_" + str(cont) 
    recommender_class = RP3betaRecommender

    data_loader = DataIO(folder_path = output_folder_path)
    search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
    best_parameters_RP3beta = search_metadata["hyperparameters_best"]

    print("Modello " + str(cont))
    model = RP3betaRecommender(URM_train_validation + URM_test)
    model.fit(**best_parameters_RP3beta)

    submissionModels[cont] = model
    cont = cont + 1
  return submissionModels
  
  



def prepare_submission(ratings: pd.DataFrame, users_to_recommend: np.array, urm_train: sps.csr_matrix, groups, models): #, recommender: object = None):
  #  users_ids_and_mappings = ratings[ratings.user_id.isin(users_to_recommend)][["user_id"]].drop_duplicates()
   # items_ids_and_mappings = ratings[["item_id", "mapped_item_id"]].drop_duplicates()
    
   # mapping_to_item_id = dict(zip(ratings.mapped_item_id, ratings.item_id))
    
    
    recommendation_length = 10
    submission = []
    for user_id in users_to_recommend:
        i = 0
        for group in groups.values():
          if user_id in group:
            #mapped_user_id = row.mapped_user_id
            
            recommendations = models[i].recommend(user_id,
                                                    recommendation_length,
                                                    )
            
            submission.append((user_id, [map_new_into_old[item_id] for item_id in recommendations]))
            break
          else: 
            i = i+1

          
    return submission


def write_submission(submissions):
    with open("./submission1.csv", "w") as f:
        f.write(f"{'user_id'},{'item_list'}\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")

[   0    1    2 ... 7944 7945 7946]


In [ ]:
#Create Submission

#submissionModels = train_on_all()
submissionModels = subissionModels
submission = prepare_submission(URM_all_dataframe, utenti, URM_train + URM_validation + URM_test, groups, subissionModels)

            
write_submission(submission)
submission

In [ ]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

recommender2 = RP3betaRecommender(URM_train_validation)


recommender2.fit(topK=121, alpha=0.5727683561350779, beta=0.0, normalize_similarity=False)
#recommender2.fit(topK=289, alpha=0.5074099391203667, beta=0.009881652632651967, normalize_similarity=False)
#recommender2.fit(topK=92, alpha=0.4669466039587683, beta=0.004657209415055234, normalize_similarity=False)


result_dict, _ = evaluator_test.evaluateRecommender(recommender2)
result_dict

RP3betaRecommender: URM Detected 54 (0.68 %) cold users.
RP3betaRecommender: URM Detected 2004 (7.72 %) cold items.
EvaluatorHoldout: Processed 21 ( 100.00% ) in 0.03 sec. Users per second: 735


{10: {'ARHR': 0.5367535903250189,
  'AVERAGE_POPULARITY': 0.15082633053221287,
  'COVERAGE_ITEM': 0.005081809432146295,
  'COVERAGE_ITEM_CORRECT': 0.0012704523580365737,
  'COVERAGE_USER': 1.0,
  'COVERAGE_USER_CORRECT': 0.8095238095238095,
  'DIVERSITY_GINI': 0.0035486969656972735,
  'DIVERSITY_HERFINDAHL': 0.9873469387755102,
  'DIVERSITY_MEAN_INTER_LIST': 0.9171428571428571,
  'F1': 0.03904218955379711,
  'HIT_RATE': 1.8095238095238095,
  'MAP': 0.08091269841269841,
  'MRR': 0.36746031746031743,
  'NDCG': 0.04570025675708339,
  'NOVELTY': 0.004424396946752545,
  'PRECISION': 0.18095238095238098,
  'PRECISION_RECALL_MIN_DEN': 0.18095238095238098,
  'RECALL': 0.02188168515759102,
  'ROC_AUC': 0.46024943310657596,
  'SHANNON_ENTROPY': 6.715765682648257}}